In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import (
    RandomForestRegressor, 
    AdaBoostRegressor, 
    ExtraTreesRegressor, 
    GradientBoostingRegressor, 
    HistGradientBoostingRegressor
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [ ]:
X, y = np.load("X_train.npy"), np.load("y_train.npy")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

models = [
    RandomForestRegressor, 
    AdaBoostRegressor, 
    ExtraTreesRegressor, 
    GradientBoostingRegressor, 
]
for i, model in enumerate(models):
    regressor = model(n_estimators=128, random_state=42)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    print("Model: ", i, "MSE:", mean_absolute_error(y_pred, y_test))

In [3]:
regressor = GradientBoostingRegressor(n_estimators=128, random_state=42)
regressor.fit(X, y)

In [ ]:
X, y = np.load("X_test.npy"), np.load("y_test.npy")

y_pred = regressor.predict(X)
df = pd.DataFrame({
    "Id": [i for i in range(len(y_pred))],
    "Predicted": y_pred,
})
df.to_csv("out.csv", index=False, sep=",")